In [28]:
import numpy as np
import pandas as pd
import plotly.offline as plot
import plotly.plotly as py
import plotly.graph_objs as go
plot.offline.init_notebook_mode(connected=True)

In [20]:
header = ['user_id', 'item_id', 'rating', 'timestamp']
df = pd.read_csv('/home/user/Downloads/ml-100k/u.data', sep='\t', names=header)

In [21]:
n_users = df.user_id.unique().shape[0]
n_items = df.item_id.unique().shape[0]
print ('Number of users = ' + str(n_users) + ' | Number of movies = ' + str(n_items))  

Number of users = 943 | Number of movies = 1682


In [22]:
#from sklearn import cross_validation as cv
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(df, test_size=0.25)
#train_data, test_data = cv.train_test_split(df, test_size=0.25)

In [23]:
train_data_matrix = np.zeros((n_users, n_items))
for line in train_data.itertuples():
    train_data_matrix[line[1]-1, line[2]-1] = line[3]  
test_data_matrix = np.zeros((n_users, n_items))
for line in test_data.itertuples():
    test_data_matrix[line[1]-1, line[2]-1] = line[3]    

In [32]:
test_data_matrix[9][711]

0.0

In [24]:
from sklearn.metrics.pairwise import pairwise_distances
user_similarity = 1-pairwise_distances(train_data_matrix, metric='cosine')

In [25]:
def predict(ratings, similarity, type='user'):
    if type == 'user':
        mean_user_rating = ratings.mean(axis=1)
        #You use np.newaxis so that mean_user_rating has same format as ratings
        ratings_diff = (ratings - mean_user_rating[:, np.newaxis]) 
        pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif type == 'item':
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])     
    return pred
user_prediction = predict(train_data_matrix, user_similarity, type='user')
user_prediction.min()

-0.2463409274233076

In [26]:
from sklearn.metrics import mean_squared_error
from math import sqrt
def rmse(prediction, ground_truth):
    prediction = prediction[ground_truth.nonzero()].flatten() 
    ground_truth = ground_truth[ground_truth.nonzero()].flatten()
    return sqrt(mean_squared_error(prediction, ground_truth))
print('User-based CF RMSE: ' + str(rmse(user_prediction, test_data_matrix)))

User-based CF RMSE: 2.9672004447309415


In [30]:
prediction = user_prediction[test_data_matrix.nonzero()].flatten() 
ground_truth = test_data_matrix[test_data_matrix.nonzero()].flatten()
data = [go.Histogram(x=ground_truth-prediction)]
layout = go.Layout(
    title='FREQUENCY OF RESIDUALS',
    xaxis=dict(
        title='RESIDUALS'
    ),
    yaxis=dict(
        title='FREQUENCY'
    )
)
fig = go.Figure(data=data, layout=layout)
plot.iplot(fig, filename='histogram')